# MP305 Activity Networks Michael Tuite

## Overview
A project consists of $N$ activities $A,B,C,....$ each taking some time to complete which obey precedence relations e.g. $C$ takes place after $A$ etc.

In this Python file activities are described by a set `Act` e.g.  `Act=set(["A","B","C"])` where `"A"`, `"B"` and `"C"` are just string names for activities such as `"First Job"`, `"Tea Break"`, ...etc

The time taken to complete the activities is described by a Python dictionary called `Time` e.g. `Time["First Job"]=3`

Precedence relations are described by a Python dictionary called `Prec` with entries giving the set of preceeding activities e.g.  `Prec["First Job"]=set()` if no preceeding activity exists or `Prec["Tea Break"]={"First Job"}` if `"Tea Break"` must take place after the first job has been completed.


In [ ]:
    import networkx as nx 
    import matplotlib.pyplot as plt

##  The `Activity` Python function
This takes the given activities set `Act`, the dictionary `Time` and the precedence relations dictionary `Prec` and relabels the activities according to the Fulkerson algorithm from 0 to $N+1$ where the START has label $0$ and the FINISH has label $N+1$.  

A list `G` of arcs `(i,j)` describing the Activity Newtork $G$ with vertices $(i,j)$  labelled by $0,1,2,.. N+1$ is produced which describes the network. A list `T` giving the completion times for the activities is also produced.

The Activity Network is drawn but note that it may not always shown in its simplest form.

In [ ]:
def Activity (Act, Time, Prec):
    global G,T,Alist,FinInd,LabelAi
    FinInd = len(Act) + 1 
    T=[0 for i in range(FinInd+1)]
    LabelAi=dict();LabeliA=dict()
    LabelAi[0] = "Start"; LabeliA["Start"] = 0
    LabelAi[FinInd] = "Finish"; LabeliA["Finish"] = FinInd
    NewP=dict()
    for AA in Act:
        NewP[AA] = Prec[AA]
    Acurrent = Act
    iA=0
    Alist = [[0]]
    while (Acurrent != set()):
        NewA = set()
        for Ai in Acurrent:
            if NewP[Ai] == set():
                NewA = NewA | set([Ai])
        NewAlist = []
        inew=1
        for An in NewA:
            LabelAi[inew + iA] = An;  LabeliA[An] = inew + iA
            NewAlist = NewAlist + [inew + iA]
            T[inew + iA] = Time[An]
            inew=inew+1
        Alist = Alist + [NewAlist] 
        iA = iA + len(NewA)
        Acurrent = Acurrent - NewA
        for Ai in Act:
            NewP[Ai] = NewP[Ai] - NewA
    Alist = Alist + [[FinInd]]
    G = set()
    FinNode = set([i for i in range(1,FinInd)])
    for AA in Act:
        iAA = LabeliA[AA]
        if Prec[AA] == set():
            G = G | set([(0,iAA)])
        else:
            for AB in Prec[AA]:
                iAB = LabeliA[AB]
                FinNode = FinNode - set([iAB])
                G = G | set([(iAB,iAA)])
    for iA in FinNode:
        G = G | set([(iA,FinInd)])
    print("The Activities have been relabeled with completion times:")
    for i in range(0, FinInd + 1):
        print("Activity ", LabelAi[i], " is activity ", i, " with completion time ", T[i])
    print("")
    print("The Activity Network is decribed by: ", G)
    plt.figure(figsize=(6, 6))
    G_Graph=nx.DiGraph()
    for arc in G:
        i,j=arc
        G_Graph.add_edge(i,j)
    Gh=20; Gw=200
    Gpos={0:(0,Gh/2),FinInd:(Gw,Gh/2)}
    ncol=len(Alist)-2
    delx=Gw/(ncol+1)
    for i in range(1,ncol+1):
        Ai=Alist[i]
        nAi=len(Ai)
        dely=Gh/(nAi+1)
        for j in range(nAi):
            Aij=Ai[nAi-j-1]
            Gpos[Aij]=(i*delx,(1+j)*dely)
    nx.draw(G_Graph,pos=Gpos, with_labels=True,node_size=1000,font_size=15,node_color='yellow',arrowsize=30)
    #print("")
   # nx.draw(G_Graph,labels=LabelAi,pos=Gpos, with_labels=True,node_size=1000,font_size=15,node_color='yellow',arrowsize=30)

# Example1: The Chemical Process example discussed in class
 Define the set of activities, completion times and precedence relations

In [ ]:
Act=set(["A","B","C","D","E","F","G"])

Time=dict()

Time["A"]=2; Time["B"]=1; Time["C"]=3; Time["D"]=3
Time["E"]=2; Time["F"]=1; Time["G"]=3

Prec=dict()

Prec["A"]=set(); Prec["B"]=set(); Prec["C"]=set()
Prec["D"]={"A","B"}
Prec["E"]={"B","C","D"}
Prec["F"]={"A","D"}
Prec["G"]={"E","F"}

In [ ]:
Act

In [ ]:
Time

In [ ]:
Prec

In [ ]:
Activity(Act,Time,Prec)

## The function `CritPath` 
This computes the critical path, the minimum project completion time, the earliest and  latest starting times and the float for each activity for an activity network `G` with completion times `T`.

In [ ]:
def CritPath (G, T):
    global FinInd,e,l,f,In,Out
    In=[[] for i in range(0, FinInd + 1)]
    Out=[[] for i in range(0, FinInd + 1)]
    for arc in G:
        i,j=arc
        In[j] = In[j]  + [i]
        Out[i]  = Out[i] + [j]

    # Forward scan for earliest starting times
    e=[0 for i in range(FinInd + 1)]
    p=[0 for i in range(FinInd + 1)]
    for j in range(1, FinInd + 1):
        #v = In[j][1]
        #e[j] = e[v] + T[v]
        #p[j] = v
        for u in In[j]:
            newe = e[u] + T[u]
            if e[j] <= newe:
                e[j] = newe
                p[j] = u

    # Backward scan for critical path
    cpath = [FinInd]
    start = FinInd
    while (not(start == 0)):
        start = p[start]
        cpath = [start] + cpath

    TTotal = 0
    for j in range(1, FinInd):
        TTotal = TTotal + T[j]
    print("The Total amount of work to be completed is:", TTotal)
    print("The Critical Path is:", cpath)
    print("The Minimum Completion Time with sufficient workers is:", e[FinInd])
    
    # latest starting times
    l=[0 for i in range(FinInd+1)] 
    f=[0 for i in range(FinInd+1)] 
    l[FinInd]=e[FinInd]
    for i in range(FinInd-1, -1, -1):
        l[i]=min([l[j] for j in Out[i]])-T[i]
        f[i]=l[i]-e[i]
    data=[["    Activity", "Earliest", "Latest", "Float"]]
    data=data +[[i,e[i],l[i],f[i]] for i in range(FinInd+1)]
    for row in data:
        print("{: >10}{: >10}{: >10}{: >10}".format(*row))

In [ ]:
CritPath(G,T)

## The function `CritSchedule` 
This implements the Critical Path Schedule for an acivity network `G` with times `T` by assigning tasks to `Nw` workers.  The output describes the assignment of activities to each worker together with a graphical display of the history over time.


This function calls the functions `RankSet1`, which ranks a set of tasks (according to latest starting times) and `GraphSchedule` which produces a graphic for the final schedule

In [ ]:
def RankSet1 (vset):
    global l
    vlist=list(vset)
    nlist = len(vlist)
    for i in range(nlist):
        for j in range(i,nlist):
                vi = vlist[i];  vj = vlist[j]
                ri = l[vi]; rj = l[vj]
                if rj < ri or (ri == rj and vj < vi):
                    vlist[i],vlist[j]=vlist[j],vlist[i]
    return(vlist)

In [ ]:
def GraphSchedule (Wseq,Time):
    import matplotlib.pyplot as plt
    from matplotlib.patches import Rectangle
    Nw=len(Wseq)
    colorw={1:"green",2:"yellow",3:"orange",4:"blue",5:"red"}
    fig = plt.figure()
    ax = fig.add_subplot() 
    Tmax=0
    for nw in range(1,Nw+1):
        Wseqn=Wseq[nw]
        AW=int(len(Wseqn)/4) # no of tasks for a worker
        y0=nw-1
        for iw in range(AW):
            t0,Aiw,t1=Wseqn[0+4*iw:3+4*iw]
            x0=t0
            width=t1-t0
            Tmax=max(Tmax,t1)
            rect = Rectangle((x0, y0), width, 1, ec ='black',lw = 2,color=colorw[nw]) 
            ax.add_patch(rect) 
            ax.text(x0+width/2, y0+1/2, Aiw,size=15)
        ax.text(-Time/3, y0+1/2, "Worker "+str(nw),size=15)
    
    # tickmarks and axes range and labels
    ax.set_xticks([k for k in range(Time+1)])
    ax.set_yticks([])
    plt.xlim([0, Time]) 
    plt.ylim([0, Nw])
    plt.xlabel('Time',size=15)
    plt.show()

In [ ]:
def CritSchedule (G, T, Nw):
#
# This function finds the optimal schedule according to the critical path scheduling algorithm for Nw workers. 
# Any idle worker is always assigned an activity if possible.
#
    global FinInd,In,Out,e,workertaskseq
    CritPath(G, T)
    print("")
# Total time for all activities
    Tot = 0
    for k in range(1, FinInd):
        Tot = Tot + T[k]
    Topt = max(Tot / Nw, e[FinInd])
# sequences of tasks undertaken by worker i
    workertaskseq=dict()
    for i in range(1, Nw + 1):
        workertaskseq[i] = []
#set of workers available
    availset = set([k for k in range(1,Nw+1)])
#worker assigned to activity k
    Activity=dict()
#
# Initial activities Done, Current and Next
#
    Current = set(); Done = set(); Next = set(Out[0]); Time = 0
#
# The Time left for activities
#
    Tleft=dict()
    for i in range(0, FinInd + 1):
        Tleft[i] = T[i]

    while (len(Done) != FinInd-1):
#
# Assign activities to available workers:
#
        Nextlist = RankSet1(set(Next))
        noc = len(Current)
        for i in range(1, min(Nw - noc, len(Next)) + 1):
            nextjob = Nextlist[i-1]
            workeri = list(availset)[0]
            Activity[nextjob] = workeri
            availset = availset - set([workeri])
            workertaskseq[workeri] = workertaskseq[workeri] + [Time, nextjob]
            Current = Current | set([nextjob])
            Next = Next - set([nextjob])
        print("Time=",Time,"Current activities=",Current)
        print("")
 # times to complete current tasks
        Tset = set()
        for v in Current:
            Tset = Tset | set([Tleft[v]])
        DelT = min(tuple(Tset))
        Time = Time + DelT
        for v in Current:
            Tleft[v] = Tleft[v] - DelT
            if Tleft[v] == 0:
                print("Time=",Time,"Activity",v,"completed")
                workerv = Activity[v]
                workertaskseq[workerv] = workertaskseq[workerv] +[Time, v]
                availset = availset | set([workerv])
                Done = Done | set([v])
                Current = Current - set([v])
                for w in Out[v]:
                    if set(In[w]) - Done == set():
                        Next = Next | set([w]) - set([FinInd])
    if Time == Topt:
        print("")
        print("Optimal Schedule Found for",Nw,"Workers!")
        if Topt == e[FinInd]:
            print("The Minimal Completion Time of",e[FinInd], "is achieved for",Nw," Workers!")
    GraphSchedule(workertaskseq,Time)

## Example 1 
Find Critical Schedule for first example in class for 1,2,3 workers

In [ ]:
CritSchedule(G,T,1)

In [ ]:
CritSchedule(G,T,2)

In [ ]:
CritSchedule(G,T,3)

## The function `ProtSchedule`  
This implements the Protection Scheme Schedule for assigning tasks to `Nw` workers. The output describes the assignement of activities to each worker together with a graphical display of the history over time. 

This function calls RankSet2 which ranks  a set of tasks according to earliest + lastest starting times

In [ ]:
def RankSet2 (vset):
    global l,e
    vlist=list(vset)
    nlist = len(vlist)
    for i in range(nlist):
        for j in range(i,nlist):
                vi = vlist[i];  vj = vlist[j]
                ri = e[vi]+l[vi]; rj =e[vj]+ l[vj]
                if rj < ri or (ri == rj and vj < vi):
                    vlist[i],vlist[j]=vlist[j],vlist[i]
    return(vlist)

In [ ]:
def ProtSchedule (G, T, Nw):
#
# This function finds the optimal schedule according to the critical path scheduling algorithm for Nw workers. 
# Any idle worker is always assigned an activity if possible.
#
    global FinInd,In,Out,e,workertaskseq
    CritPath(G, T)
    print("")
# Total time for all activities
    Tot = 0
    for k in range(1, FinInd):
        Tot = Tot + T[k]
    Topt = max(Tot / Nw, e[FinInd])
# sequences of tasks undertaken by worker i
    workertaskseq=dict()
    for i in range(1, Nw + 1):
        workertaskseq[i] = []
#set of workers available
    availset = set([k for k in range(1,Nw+1)])
#worker assigned to activity k
    Activity=dict()
#
# Initial activities Done, Current and Next
#
    Current = set(); Done = set(); Next = set(Out[0]); Time = 0
#
# The Time left for activities
#
    Tleft=dict()
    for i in range(0, FinInd + 1):
        Tleft[i] = T[i]
    AllRank=RankSet2(set([i for i in range(1,FinInd+1)]));
    while (len(Done) != FinInd-1):
#
# Assign activities to available workers:
#
        Nextlist = RankSet2(set(Next))
        noc = len(Current)
        for i in range(1, min(Nw - noc, len(Next)) + 1):
            nextjob = Nextlist[i-1]
            workeri = list(availset)[0]
            if nextjob==AllRank[0]: 
                Activity[nextjob] = workeri
                availset = availset - set([workeri])
                workertaskseq[workeri] = workertaskseq[workeri] + [Time, nextjob]
                Current = Current | set([nextjob])
                Next = Next - set([nextjob])
                AllRank=RankSet2(set(AllRank) - set([nextjob]))
        print("Time=",Time,"Current activities=",Current)
        print("")
 # times to complete current tasks
        Tset = set()
        for v in Current:
            Tset = Tset | set([Tleft[v]])
        DelT = min(tuple(Tset))
        Time = Time + DelT
        for v in Current:
            Tleft[v] = Tleft[v] - DelT
            if Tleft[v] == 0:
                print("Time=",Time,"Activity",v,"completed")
                workerv = Activity[v]
                workertaskseq[workerv] = workertaskseq[workerv] +[Time, v]
                availset = availset | set([workerv])
                Done = Done | set([v])
                Current = Current - set([v])
                for w in Out[v]:
                    if set(In[w]) - Done == set():
                        Next = Next | set([w]) - set([FinInd])
    if Time == Topt:
        print("")
        print("Optimal Schedule Found for",Nw,"Workers!")
        if Topt == e[FinInd]:
            print("The Minimal Completion Time of",e[FinInd], "is achieved for",Nw," Workers!")
    GraphSchedule(workertaskseq,Time)

## Example 1 
Find the Protected Schedule for first example in class for 2,3 workers

In [ ]:
ProtSchedule (G, T, 2)

# Lecture 8 example 

In [ ]:
Act=set([1,2,3,4,5,6])

Time=dict()

Time[1]=2; Time[2]=2; Time[3]=8; Time[4]=5
Time[5]=5; Time[6]=8

Prec=dict()

Prec[1]=set(); Prec[2]=set(); Prec[3]=set()
Prec[4]={1}
Prec[5]={1}
Prec[6]={4,5}

In [ ]:
Activity(Act, Time, Prec)

## Critical Schedule

In [ ]:
CritSchedule (G, T, 2)

## Protection Schedule

In [ ]:
ProtSchedule (G, T, 2)